# Motion Sensor Controller

This code controls the motion sensors and sends data to the web appplication.

Spacing specifies the distance between the sensors, so that the velocity may be calculated (not important to motion detection algorithm).

In [ ]:
from pynq.overlays.base import BaseOverlay
import time
base = BaseOverlay('base.bit')
spacing = 1.8

This cell loads config data from this directory, most notibly the server address.

In [ ]:
import json
config = json.load(open('config.json'))

This is the function responsible for sending data to the server. It runs concurrently in a separate thread, to allow the sensor code to run in parallel.

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from requests_futures.sessions import FuturesSession
executor = ThreadPoolExecutor(max_workers=4)
server = config['server']
session = FuturesSession(executor=executor)
f_params = session.post(server + '/register?lot=1')
def log_to_server(direc):
    params = f_params.result().json()
    params[direc] = 1
    session.post(server + "/report", json=params)

## Setting up the sensors and outputs

Set up the sensors and debugging outputs.

1. You should connect the **first PIR sensor for the enterance** to `G1` on `PMODA`.
2. You should connect the **first PIR sensor for the exit** to `G2` on `PMODA`.
3. Optionally, you may connect the LED bar to `G4` on `PMODB` (useful for debugging).
4. Optionally, you may connect a buzzer to `G1` on the Arduino shield (useful for debugging).


In [ ]:
from pynq.lib.pmod import *
from pynq.lib.arduino import Grove_Buzzer as A_Grove_Buzzer, ARDUINO_GROVE_G1
sensors = [Grove_PIR(base.PMODA, port) for port in (PMOD_GROVE_G1, PMOD_GROVE_G2)]
bar = Grove_LEDbar(base.PMODB, PMOD_GROVE_G4)
buzz = A_Grove_Buzzer(base.ARDUINO, ARDUINO_GROVE_G1)

## Detecting Movement & Direction

These are functions to play ascending and descending dubugging tones:

In [ ]:
def tone_asc():
    buzz.play_tone(494, 428)
    buzz.play_tone(659, 428)
    
def tone_desc():
    buzz.play_tone(659, 428)
    buzz.play_tone(494, 428)

The actual code -- pretty simple: if something triggered the other sensor (change in output from PIR sensor) in the past 10 seconds, then trigger the direction of the motion.

In [ ]:
detect = [0 for s in sensors]
last = [s.read() for s in sensors]
while True:
    display = 0
    for m, i, s in zip([0b1111100000, 0b0000011111], [0, 1], sensors):
        r = s.read()
        clock = time.process_time()
        display |= r and m
        if r != last[i]:
            # change in sensor output
            last[i] = r
            if r and detect[not i]:
                executor.submit([tone_desc, tone_asc][i])
                log_to_server(["exit", "enter"][i])
                print("in" if i else "out", "|", spacing/(clock - detect[not i]), "m/s")
                detect[not i] = 0
            elif r:
                detect[i] = clock
    detect = [0 if clock - 4 > v else v for v in detect]
    bar.write_binary(display)